# Part 1 Prompt

Ok, so it's similar to Day 2, only now the instructions take a variable amount of space:

 - 1 and 2 take 4 spaces (opcode, 2 parameters, result index)
 - 3 takes 2 spaces, (opcode, result index) and has a notion of an "input value" separate from the code
 - 4 takes 2 spaces, (opcode, 1 parameter) and has a notion of an "output value" separate from the code

So now the "header" part of the instruction has both the opcode (which function to run) and the "mode" for up to three parameters.  So I guess what I have been calling the "result index" is also a parameter under this system

I still don't understand how opcode 4 works, if it only has 1 parameter.  Immediate/position mode just means whether you use the pointer or the value of the pointer, but it doesn't explain what it means to "output" 50

Ok, I am going to interpret "input" and "output" as typical IO operations.  Meaning, we expect the first command to be 3, and we are supposed to type in `1` when that happens, then every time we encounter a 4, that should get printed out.  All of the things printed out should be 0 except the final thing

### Some Experiments

In [1]:
int("")

ValueError: invalid literal for int() with base 10: ''

So I might manually have to specify that empty means 0

In [2]:
int("02")

2

In [3]:
x = input("Enter something: ")

Enter something:  hello


In [4]:
x

'hello'

Ok good, I can still use `input` in a Jupyter Notebook

### Old Version

In [5]:
def run_list_program(program_codes):
    # loop through codes in 4-code "programs"
    current_position = 0
    while True:
        opcode = program_codes[current_position]
        if opcode == 99:
            break
        else:
            # for inputs, first we have to get their index, then we have
            # to get the values at that index
            input_1_index = program_codes[current_position+1]
            input_2_index = program_codes[current_position+2]

            input_1 = program_codes[input_1_index]
            input_2 = program_codes[input_2_index]

            # for output, we only need the index, since we are overwriting
            # whatever is there
            destination_index = program_codes[current_position+3]
            
            # compute the output
            if opcode == 1:
                output = input_1 + input_2
            elif opcode == 2:
                output = input_1 * input_2
            else:
                print("Something went wrong, got opcode", opcode)
                break
            
            # set the destination index to the output
#             print("Position", destination_index, "is now set to", output)
            program_codes[destination_index] = output
            
            # move to the next "program"
            current_position = current_position + 4

What needs to change?

 - pull out opcode but also modes of each parameter
 - get values of parameters conditionally based on position/immediate mode
 - implement opcodes 3 and 4
 - move to the next program conditionally based on length of current program

In [7]:
def get_param_mode(param_modes_str, param_num):
    # param_num is 1, 2, or 3
    if param_num <= len(param_modes_str):
        mode_code = param_modes_str[-param_num]
        if mode_code == "0":
            return "position"
        else:
            return "immediate"
    else:
        # if it's missing from the instruction, it's always "position"
        return "position"

In [8]:
def get_index_or_value(number, position_mode, program_codes):
    if position_mode == "immediate":
        return number
    else:
        return program_codes[number]

I'm confused now why there are modes for 3 parameters, if the only 3rd parameter is always an output and those will "never be in immediate mode".  Maybe this is something relevant for part 2?

In [9]:
def run_list_program(program_codes):
    # loop through codes in 4-code "programs"
    current_position = 0
    while True:
        opcode_and_param_modes = str(program_codes[current_position])
        # last two digits is opcode
        opcode = int(opcode_and_param_modes[-2:])

        if opcode == 99:
            break
        else:
            param_modes = opcode_and_param_modes[:-2]
            
            if opcode == 1 or opcode == 2:
                input_1_mode = get_param_mode(param_modes, 1)
                input_2_mode = get_param_mode(param_modes, 2)
                
                # not doing anything with this information right now, because it is
                # seemingly always position mode
                output_mode = get_param_mode(param_modes, 3)
                
                input_1_number = program_codes[current_position+1]
                input_2_number = program_codes[current_position+2]
                
                input_1 = get_index_or_value(input_1_number, input_1_mode, program_codes)
                input_2 = get_index_or_value(input_2_number, input_2_mode, program_codes)

                # for output, we only need the index, since we are overwriting
                # whatever is there
                destination_index = program_codes[current_position+3]
            
                # compute the output
                if opcode == 1:
                    output = input_1 + input_2
                elif opcode == 2:
                    output = input_1 * input_2
            
                program_codes[destination_index] = output
            
                # move to the next "program", opcodes 1 and 2 have length 4
                current_position = current_position + 4

Currently only implemented opcodes 1 and 2, testing

In [10]:
example_program = [1002,4,3,4,33]

In [11]:
run_list_program(example_program)

In [12]:
example_program

[1002, 4, 3, 4, 99]

In [13]:
another_example_program = [1101,100,-1,4,0]

In [14]:
run_list_program(another_example_program)

In [15]:
another_example_program

[1101, 100, -1, 4, 99]

Dang, they're not even going to give any tests with opcodes 3 and 4

In [16]:
def run_list_program(program_codes):
    # loop through codes in 4-code "programs"
    current_position = 0
    while True:
        opcode_and_param_modes = str(program_codes[current_position])
        # last two digits is opcode
        opcode = int(opcode_and_param_modes[-2:])

        if opcode == 99:
            break
        else:
            param_modes = opcode_and_param_modes[:-2]
            
            if opcode == 1 or opcode == 2:
                input_1_mode = get_param_mode(param_modes, 1)
                input_2_mode = get_param_mode(param_modes, 2)
                
                # not doing anything with this information right now, because it is
                # seemingly always position mode
                output_mode = get_param_mode(param_modes, 3)
                
                input_1_number = program_codes[current_position+1]
                input_2_number = program_codes[current_position+2]
                
                input_1 = get_index_or_value(input_1_number, input_1_mode, program_codes)
                input_2 = get_index_or_value(input_2_number, input_2_mode, program_codes)

                # for output, we only need the index, since we are overwriting
                # whatever is there
                destination_index = program_codes[current_position+3]
            
                # compute the output
                if opcode == 1:
                    output = input_1 + input_2
                elif opcode == 2:
                    output = input_1 * input_2
            
                program_codes[destination_index] = output
            
                # move to the next "program", opcodes 1 and 2 have length 4
                current_position = current_position + 4
            elif opcode == 3:
                user_input = int(input("Enter a number: "))
                destination_index = program_codes[current_position+1]
                program_codes[destination_index] = user_input
                current_position = current_position + 2
            elif opcode == 4:
                input_1_mode = get_param_mode(param_modes, 1)
                input_1_number = program_codes[current_position+1]
                input_1 = get_index_or_value(input_1_number, input_1_mode, program_codes)
                print(input_1)
                current_position = current_position + 2
            else:
                print("Error, encountered opcode", opcode)
                break

Ok, I'm basically going in blind here

In [17]:
part1_input_file = open("input.txt", "r")
part1_input = part1_input_file.readline()
part1_input

'3,225,1,225,6,6,1100,1,238,225,104,0,2,218,57,224,101,-3828,224,224,4,224,102,8,223,223,1001,224,2,224,1,223,224,223,1102,26,25,224,1001,224,-650,224,4,224,1002,223,8,223,101,7,224,224,1,223,224,223,1102,44,37,225,1102,51,26,225,1102,70,94,225,1002,188,7,224,1001,224,-70,224,4,224,1002,223,8,223,1001,224,1,224,1,223,224,223,1101,86,70,225,1101,80,25,224,101,-105,224,224,4,224,102,8,223,223,101,1,224,224,1,224,223,223,101,6,91,224,1001,224,-92,224,4,224,102,8,223,223,101,6,224,224,1,224,223,223,1102,61,60,225,1001,139,81,224,101,-142,224,224,4,224,102,8,223,223,101,1,224,224,1,223,224,223,102,40,65,224,1001,224,-2800,224,4,224,1002,223,8,223,1001,224,3,224,1,224,223,223,1102,72,10,225,1101,71,21,225,1,62,192,224,1001,224,-47,224,4,224,1002,223,8,223,101,7,224,224,1,224,223,223,1101,76,87,225,4,223,99,0,0,0,677,0,0,0,0,0,0,0,0,0,0,0,1105,0,99999,1105,227,247,1105,1,99999,1005,227,99999,1005,0,256,1105,1,99999,1106,227,99999,1106,0,265,1105,1,99999,1006,0,99999,1006,227,274,1105,1,99999,

In [18]:
part1_input = part1_input.strip()

In [19]:
part1_input_codes = [int(x) for x in part1_input.split(",")]

In [20]:
run_list_program(part1_input_codes)

Enter a number:  1


0
0
0
0
0
0
0
0
0
6069343


That worked, and I hate it

# Part 2 Prompt

We really are straight up implementing an operating system here huh

In [21]:
def get_value_for_input(param_modes, param_number, program_codes, current_position):
    """ Now that we have to do this for the input of more than 3 opcodes, making a
    function makes more sense
    """
    input_mode = get_param_mode(param_modes, param_number)
    input_number = program_codes[current_position + param_number]
    actual_value = get_index_or_value(input_number, input_mode, program_codes)
    return actual_value

In [29]:
def run_list_program(program_codes):
    # loop through codes in 4-code "programs"
    current_position = 0
    while True:
        opcode_and_param_modes = str(program_codes[current_position])
        # last two digits is opcode
        opcode = int(opcode_and_param_modes[-2:])

        # made the decision to indent as little as possible, short-circuit instead
        if opcode == 99:
            break
        
        param_modes = opcode_and_param_modes[:-2]
        
        # these opcodes have a first parameter
        if opcode in [1,2,4,5,6,7,8]:
            input_1 = get_value_for_input(param_modes, 1, program_codes, current_position)
        
        # these opcodes have a second parameter
        if opcode in [1,2,5,6,7,8]:
            input_2 = get_value_for_input(param_modes, 2, program_codes, current_position)

        if opcode == 1 or opcode == 2:
            """ Given two inputs and a destination index, perform some operation on 
            the two inputs and put the result in the destination index """
            destination_index = program_codes[current_position+3]

            if opcode == 1:
                output = input_1 + input_2
            elif opcode == 2:
                output = input_1 * input_2

            program_codes[destination_index] = output

            # move to the next "program", opcodes 1 and 2 have length 4
            current_position = current_position + 4
        elif opcode == 3:
            """ Here the input comes from prompting the user, then that is copied to
            the destination index """
            user_input = int(input("Enter a number: "))
            destination_index = program_codes[current_position+1]
            program_codes[destination_index] = user_input
            current_position = current_position + 2
        elif opcode == 4:
            """ Given one input (retrieved above), print it """
            print(input_1)
            current_position = current_position + 2
        elif opcode == 5 or opcode == 6:
            """ Based on the first input, conditionally sets current_position to
            the value of the second input """
            if opcode == 5:
                should_jump = input_1 != 0
            elif opcode == 6:
                should_jump = input_1 == 0
                
            if should_jump:
                current_position = input_2
            else:
                current_position = current_position + 3
        elif opcode == 7 or opcode == 8:
            """ Based on a comparison of the first and second inputs, sets
            destination index to 1 or 0 """
            if opcode == 7:
                comparison_true = input_1 < input_2
            elif opcode == 8:
                comparison_true = input_1 == input_2
                
            destination_index = program_codes[current_position + 3]
            if comparison_true:
                program_codes[destination_index] = 1
            else:
                program_codes[destination_index] = 0
                
            current_position = current_position + 4

        else:
            print("Error, encountered opcode", opcode)
            break

In [30]:
part1_input_codes = [int(x) for x in part1_input.split(",")]
run_list_program(part1_input_codes)

Enter a number:  1


0
0
0
0
0
0
0
0
0
6069343


In [31]:
run_list_program([3,9,8,9,10,9,4,9,99,-1,8])

Enter a number:  8


1


In [32]:
run_list_program([3,9,7,9,10,9,4,9,99,-1,8])

Enter a number:  8


0


In [33]:
run_list_program([3,3,1108,-1,8,3,4,3,99])

Enter a number:  8


1


In [34]:
run_list_program([3,3,1107,-1,8,3,4,3,99])

Enter a number:  8


0


In [35]:
run_list_program([3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9])

Enter a number:  8


1


In [36]:
run_list_program([3,3,1105,-1,9,1101,0,0,12,4,12,99,1])

Enter a number:  8


1


In [37]:
run_list_program([3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99])

Enter a number:  7


999


In [38]:
run_list_program([3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99])

Enter a number:  8


1000


In [39]:
run_list_program([3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99])

Enter a number:  999


1001


In [40]:
part2_input = part1_input[:]

In [41]:
part2_input_codes = [int(x) for x in part2_input.split(",")]

What is the diagnostic code for system ID 5?

In [42]:
run_list_program(part2_input_codes)

Enter a number:  5


3188550
